# Time Series Prediction

**Objectives**

1. Build a linear, DNN and CNN model in Keras
2. Build a simple RNN model and a multi-layer RNN model in Keras

In this lab, we'll start with a linear, DNN and CNN model.

Since the features of our model are sequential in nature, we'll next look at how to build various RNN models in Keras. We'll start with a simple RNN model and then see how to create a multi-layer RNN in Keras.